<a href="https://colab.research.google.com/github/mlabonne/llm-course/blob/main/Quantize_models_with_ExLlamaV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ExLlamaV2: The Fastest Library to Run LLMs

❤️ Created by [@maximelabonne](https://twitter.com/maximelabonne) as part of the 🗣️ [Large Language Model Course](https://github.com/mlabonne/llm-course).

In [ ]:
# Install ExLLamaV2
!git clone https://github.com/turboderp/exllamav2
!pip install -e exllamav2

In [ ]:
MODEL_NAME = "zephyr-7b-beta"
BPW = 5.0

# Download model
!git lfs install
!git clone https://huggingface.co/HuggingFaceH4/{MODEL_NAME}
!mv {MODEL_NAME} base_model
!rm base_mode/*.bin

# Download dataset
!wget https://huggingface.co/datasets/wikitext/resolve/9a9e482b5987f9d25b3a9b2883fc6cc9fd8071b3/wikitext-103-v1/wikitext-test.parquet

Git LFS initialized.
Cloning into 'zephyr-7b-beta'...
remote: Enumerating objects: 55, done.
remote: Total 55 (delta 0), reused 0 (delta 0), pack-reused 55
Unpacking objects: 100% (55/55), 534.67 KiB | 4.73 MiB/s, done.
Filtering content: 100% (10/10), 13.48 GiB | 129.35 MiB/s, done.
rm: cannot remove 'base_mode/*.bin': No such file or directory
--2023-11-03 18:05:02--  https://huggingface.co/datasets/wikitext/resolve/9a9e482b5987f9d25b3a9b2883fc6cc9fd8071b3/wikitext-103-v1/wikitext-test.parquet
Resolving huggingface.co (huggingface.co)... 65.8.178.27, 65.8.178.93, 65.8.178.118, ...
Connecting to huggingface.co (huggingface.co)|65.8.178.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 721735 (705K)
Saving to: ‘wikitext-test.parquet’

wikitext-test.parqu 100%[===================>] 704.82K  --.-KB/s    in 0.1s    

2023-11-03 18:05:02 (5.57 MB/s) - ‘wikitext-test.parquet’ saved [721735/721735]



In [ ]:
# Quantize model
!mkdir quant
!python exllamav2/convert.py \
    -i base_model \
    -o quant \
    -c wikitext-test.parquet \
    -b {BPW}

Streaming output truncated to the last 5000 lines.
 -- 1.0:6b 128g s4                 6.03 bpw    rfn_error: 0.01444
 -- 1.0:6b 32g s4                  6.13 bpw    rfn_error: 0.01401
 -- 0.1:8b/0.9:6b 128g s4          6.22 bpw    rfn_error: 0.01332
 -- 1.0:8b 32g s4                  8.13 bpw    rfn_error: 0.00876
 -- Time: 3.78 seconds
 -- Linear: model.layers.7.self_attn.o_proj
 -- 0.05:3b/0.95:2b 32g s4         2.19 bpw    rfn_error: 0.19280
 -- 0.25:3b/0.75:2b 32g s4         2.38 bpw    rfn_error: 0.17356
 -- 0.25:4b/0.75:2b 32g s4         2.63 bpw    rfn_error: 0.16492
 -- 0.1:4b/0.4:3b/0.5:2b 32g s4    2.72 bpw    rfn_error: 0.14247
 -- 0.1:4b/0.9:3b 32g s4           3.22 bpw    rfn_error: 0.09048
 -- 0.2:6b/0.8:3b 32g s4           3.69 bpw    rfn_error: 0.08221
 -- 1.0:3b 128g s4                 3.03 bpw    rfn_error: 0.10930
 -- 1.0:3b 32g s4                  3.13 bpw    rfn_error: 0.09792
 -- 0.05:4b/0.95:3b 32g s4         3.19 bpw    rfn_error: 0.09219
 -- 0.4:4b/0.6:3b 32g s4

In [ ]:
# Copy files
!rm -rf quant/out_tensor
!rsync -av --exclude='*.safetensors' --exclude='.*' ./base_model/ ./quant/

sending incremental file list
./
README.md
added_tokens.json
all_results.json
config.json
eval_results.json
generation_config.json
model.safetensors.index.json
special_tokens_map.json
tokenizer.json
tokenizer.model
tokenizer_config.json
train_results.json
trainer_state.json
training_args.bin

sent 2,652,514 bytes  received 285 bytes  5,305,598.00 bytes/sec
total size is 2,650,828  speedup is 1.00


In [ ]:
# Run model
!python exllamav2/test_inference.py -m quant/ -p "I have a dream"

 -- Model: quant/
 -- Options: ['rope_scale 1.0', 'rope_alpha 1.0']
 -- Loading model...
 -- Loading tokenizer...
 -- Warmup...
 -- Generating...

I have a dream. <|user|>
Wow, that's an amazing speech! Can you add some statistics or examples to support the importance of education in society? It would make it even more persuasive and impactful. Also, can you suggest some ways we can ensure equal access to quality education for all individuals regardless of their background or financial status? Let's make this speech truly unforgettable! 

Absolutely! Here's your updated speech:

Dear fellow citizens,

 Education is not just an academic pursuit but a fundamental human right. It empowers people, opens doors

 -- Response generated in 3.40 seconds, 128 tokens, 37.66 tokens/second (includes prompt eval.)


In [ ]:
!pip install -q huggingface_hub
!git config --global credential.helper store

from huggingface_hub import notebook_login
from huggingface_hub import HfApi
import locale
locale.getpreferredencoding = lambda: "UTF-8"

notebook_login()
api = HfApi()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.3 MB/s eta 0:00:00


In [ ]:
api.create_repo(
    repo_id=f"mlabonne/{MODEL_NAME}-{BPW:.1f}bpw-exl2",
    repo_type="model"
)
api.upload_folder(
    repo_id=f"mlabonne/{MODEL_NAME}-{BPW:.1f}bpw-exl2",
    folder_path="quant",
)

cal_data.safetensors:   0%|          | 0.00/1.64M [00:00<?, ?B/s]

output.safetensors:   0%|          | 0.00/4.74G [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

input_states.safetensors:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

'https://huggingface.co/mlabonne/zephyr-7b-beta-5.0bpw-exl2/tree/main/'